In [1]:
import pymongo
client = pymongo.MongoClient(mongodb_uri)
db = client["schedule"]

In [12]:
import requests
import xml.etree.ElementTree as ET  # for parsing XML
import multiprocessing as mp
from datetime import datetime

classes = db["classes"]

In [10]:
for clas in classes.find({'sections': {'$exists': False}}) :
    resp = requests.get(clas["api_link"])
    if resp.status_code == 404 :
        print("Error Link not found: " + clas["name"])
        classes.delete_one({"_id": clas["_id"]})
        continue
    if resp.status_code != 200 :
        print("Error: " + clas["name"])
        print(clas["api_link"])
        continue
    clas_root = ET.fromstring(resp.content)
    try : 
        description = clas_root.find("description").text
    except :
        print("Description not found for " + clas["name"])
        print(clas["api_link"])
        description = ""
    credit_hours = clas_root.find("creditHours").text
    sections = []
    for section in clas_root.find("sections").findall("section") :
        sections.append({
            'crn': section.attrib["id"],
            'api_link': section.attrib["href"],
            'name': section.text
        })
    classes.update_one({"_id": clas["_id"]}, {"$set": {"description": description, "credit_hours": credit_hours, "sections": sections}})

Error Link not found: Grain Bioprocessing Coproducts


In [38]:
def find_validate_xml (base, search) : 
    try :
        return base.find(search).text
    except :
        return None

In [39]:
def update_sections (clas):
    for section in clas['sections'] :
        resp = requests.get(section["api_link"])
        if resp.status_code == 404 :
            print("Error Link not found: " + section["name"])
            continue
        if resp.status_code != 200 :
            print("Error: " + section["name"])
            print(section["api_link"])
            continue
            
        try :
            section_root = ET.fromstring(resp.content)
        except :
            print("Error parsing XML: " + section["crn"])
            continue
        
        status_code = find_validate_xml(section_root, "statusCode")
        part_of_term = find_validate_xml(section_root, "partOfTerm")
        section_status_code = find_validate_xml(section_root, "sectionStatusCode")
        enrollment_status = find_validate_xml(section_root, "enrollmentStatus")

        try :
            start_date_string = section_root.find("startDate").text # 2023-08-21Z
            start_date = datetime.strptime(start_date_string, "%Y-%m-%dZ")
        except :
            start_date = None
        
        try : 
            end_date_string = section_root.find("endDate").text # 2023-12-09Z
            end_date = datetime.strptime(end_date_string, "%Y-%m-%dZ")
        except :
            end_date = None

        meetings = []
        for meeting in section_root.find("meetings").findall("meeting") :

            try :
                start_string = meeting.find("start").text # 09:00 AM
                start_time = datetime.strptime(start_string, "%I:%M %p")
            except:
                start_time = None

            try : 
                end_string = meeting.find("end").text # 09:50 AM
                end_time = datetime.strptime(end_string, "%I:%M %p")
            except :
                end_time = None

            instructors = []
            for instructor in meeting.find("instructors").findall("instructor") :
                instructors.append(instructor.text)

            try : 
                type_code = meeting.find("type").attrib["code"]
            except :
                type_code = None

            meetings.append({
                'id': meeting.attrib["id"],
                'type' : find_validate_xml(meeting, "type"),
                'type_code' : type_code,
                'start_time' : start_time,
                'end_time' : end_time,
                'days' : find_validate_xml(meeting, "days"),
                'room_number' : find_validate_xml(meeting, "roomNumber"),
                'building_name' : find_validate_xml(meeting, "buildingName"),

            })

        classes.update_one(
            {"_id": clas["_id"], "sections.crn": section["crn"]}, 
            {"$set": {"sections.$.status_code": status_code, "sections.$.part_of_term": part_of_term, "sections.$.section_status_code": section_status_code, "sections.$.enrollment_status": enrollment_status, "sections.$.start_date": start_date, "sections.$.end_date": end_date, "sections.$.meetings": meetings}})

In [40]:
# list_of_classes = list(classes.find({'sections.status_code': {'$exists': False}}))
list_of_classes = list(classes.find({}))

pool = mp.Pool(mp.cpu_count())
pool.map(update_sections, list_of_classes)
pool.close()


Error Link not found: MLC
